## Problem
### Game of Life
Conway's Game of Life is a cellular automaton that is played on a grid of cells. Each cell can be either alive or dead. The game progresses in discrete steps. At each step, the following rules are applied to each cell:

- Any live cell with fewer than two live neighbours dies, as if by underpopulation.
- Any live cell with two or three live neighbours lives on to the next generation.
- Any live cell with more than three live neighbours dies, as if by overpopulation.
- Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

A cell's neighbours are those cells which are horizontally, vertically or diagonally adjacent. Most cells will have eight neighbours. We also follow wrapping conventions, so that the top row is adjacent to the bottom row, and the left column is adjacent to the right column.
Furthermore, we do not update the grid in place. This means we first compute the next state of the grid using the current state and then update the grid with the new state. Notably, this is often done using two arrays, one for the current state and one for the next state. This is because the next state of a cell depends on the current state of the cell and its neighbours. If we were to update the grid in place, then we would be using the new state of a cell to compute the new state of its neighbours, which is not what we want.

### Our parametrisation
For this challenge, we make a small change to the rules above. Instead of having a fixed number of neighbours for each of the rules, we allow the number of neighbours to be a parameter. In particular, we have three parameters A, B, and C and the rules are as follows:

- Any live cell with fewer than A live neighbours dies, as if by underpopulation.
- Any live cell with between A or B live neighbours lives on to the next generation (inclusive).
- Any live cell with more than B live neighbours dies, as if by overpopulation.
- Any dead cell with exactly C live neighbours becomes a live cell, as if by reproduction.

### Input
Your input will be passed in via standard in and will consist of several lines. The first line will be:

w h n m A B C, where:

- w is the width of the grid
- h is the height of the grid
- n is the number of steps to simulate
- m is the output frequency, i.e. the number of steps between each output
Then, A, B, and C, indicate the numbers for the rules mentioned above
We guarantee that the parameters will have the following bounds:

The next h lines will indicate the initial state. Each line will have w characters, each of which will be either . (dead) or # (alive).

### Output
The output must be the state of the grid, every m steps, for n steps. The output must be in the same format as the input. For instance, if n = 1001 and m = 250, we output the following:

The result after the first iteration
The result after the 251st iteration
The result after the 501st iteration
The result after the 751st iteration
The result after the 1001st iteration
You must always output the final state of the grid, but do not output it twice. Furthermore, the state after the first iteration must also always be printed, regardless of the value of m.

In [180]:
#Creating the print grid function
def printGrid(grid):
    bigGrid = []
    for i in grid:
        bigGrid.append(''.join(i))
    print('\n'.join(bigGrid))

#Creating the main game function
def conwaysGameOfLife(w, initial_grid):
    #Converting the initial grid to a list form
    #Separating into seperate rows, every 10 characters
    grid_splits = [initial_grid[i:i+w] for i in range(0, len(initial_grid), w)]
    #Seperating each row into different columns
    griddy = []
    for i in grid_splits:
        griddy.append(list(i))
    return griddy

In [182]:
printGrid(conwaysGameOfLife(10,'...........#.........#.......................#..#........##.........#.#.............................'))

..........
.#........
.#........
..........
.....#..#.
.......##.
........#.
#.........
..........
..........


In [4]:
test_grid = conwaysGameOfLife(10, 10, 1000000, 100000000, 2, 3, 3, '...........#.........#.......................#..#........##.........#.#.............................')

To make decisions on whether the cell will be dead or alive, we need the number of neighbors it has. To achieve this we need to iterate first get how many live neighbors a cell has. This can be done by iterating through each element in the list and keep track of row and column number

In [5]:
test_grid

[['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '#', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '#', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '#', '.', '.', '#', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '#', '#', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.']]

In [6]:
#Case 1
test_grid[0][10]

IndexError: list index out of range

In [7]:
#Case 2
test_grid[0][-11]

IndexError: list index out of range

Because each side of the grid is wrapped we have a problem with edge cells (Case 1). Hence we need to account for the case where our iterating variable is greater than the range of the index. This is only problem when we increment as the negative range will never be reached (Case 2)

In [8]:
#Neighbour mappings are: 

#  left-u |   up  | right-u
#  left-c | index | right-c
#  left-d |  down | right-d 

In [183]:
def cellHealth(value_arr):
    neighbor_result = []
    for i in value_arr:
        output = []
        for j in i:
            if j=='#':
                output.append(1)
            else:
                output.append(0)
        neighbor_result.append(output)
    return neighbor_result

def neighbour_checker(test_grid, i, j, w, h):
    i_down = i+1
    j_right = j+1

    if i == w-1:
        i_down = 0
    if j == h-1:
        j_right = 0
    
    #the current index is  [i][j]
    #Checking left-u neighbor
    lu = test_grid[i-1][j-1]
    #Checking left-c neighbour
    lc =test_grid[i][j-1]
    #Checking left_d neighbour
    ld = test_grid[i_down][j-1]

    #Checking up neighbour
    u = test_grid[i-1][j]
    #Checking down neighbour
    d = test_grid[i_down][j]

    #Checking right-u neighbour
    ru = test_grid[i-1][j_right]
    #Checking right-c neighbour
    rc = test_grid[i][j_right]
    #Checking right-d neighbour
    rd = test_grid[i_down][j_right]
    return cellHealth([[lu, lc , ld], [u, d], [ru, rc, rd]])

In [184]:
def neighbour_sum(lists):
    neighbors = 0
    for i in lists:
        neighbors += sum(i)
    return neighbors

def neighbour_calc(grid, i, j, w, h):
    return neighbour_sum(neighbour_checker(grid, i, j, w, h))

In [187]:
test2_grid = conwaysGameOfLife(10, '#.......#..#.........#.......................#..#........##.........#.#.............................')

In [189]:
neighbour_checker(test2_grid, 1, 1, 10, 10)

[[1, 0, 0], [0, 1], [0, 0, 0]]

In [191]:
neighbour_calc(test2_grid, 0, 9, 10, 10)

2

In [192]:
test2_grid

[['#', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
 ['.', '#', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '#', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '#', '.', '.', '#', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '#', '#', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.']]

## Using the functions to iterate through grid and make decisions

In [193]:
#Any live cell with fewer than two live neighbours dies, as if by underpopulation.
#Any live cell with two or three live neighbours lives on to the next generation.
#Any live cell with more than three live neighbours dies, as if by overpopulation.
#Any dead cell with exactly three live neighbours becomes a live cell, as if by reproduction.

In [194]:
def cellAliveOrDead(grid, i, j, A, B, C, w, h):
    cellValue = grid[i][j]
    cellState = False
    if cellValue == '#':
        cellState = True
    else:
        cellState = False
    neighbours = neighbour_calc(grid, i, j, w, h)
    
    #For live cells
    if neighbours < A and cellState == True:
        return '.'
    if (neighbours == A or neighbours == B) and cellState == True:
        return '#'
    if neighbours > B and cellState == True:
        return '.'
    
    #For dead cells
    if neighbours == B and cellState == False:
        return '#'
    #Condition for dead cells to remain dead
    return '.'

# Doing a test

In [195]:
new_test = conwaysGameOfLife(10, '...........#.........#.......................#..#........##.........#.#.............................')

In [196]:
new_test

[['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '#', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '#', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '#', '.', '.', '#', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '#', '#', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
 ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.']]

In [198]:
neighbor_grid = []
for i in range(0, h):
    temp_neighbor_out = []
    for j in range(0, w):
        temp_neighbor_out.append(neighbour_calc(new_test, i, j, w, h))
    neighbor_grid.append(temp_neighbor_out)

In [199]:
neighbor_grid

[[1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
 [2, 1, 2, 0, 0, 0, 0, 0, 0, 0],
 [2, 1, 2, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
 [0, 0, 0, 0, 1, 0, 2, 3, 2, 2],
 [0, 0, 0, 0, 1, 1, 2, 3, 3, 3],
 [1, 1, 0, 0, 0, 0, 1, 3, 2, 3],
 [0, 1, 0, 0, 0, 0, 0, 1, 1, 2],
 [1, 1, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [200]:
w = 10
h = 10

In [202]:
new_grid = []
for i in range(0, h):
    new_grid_temp = []
    for j in range(0, w):
        new_grid_temp.append(cellAliveOrDead(new_test, i, j, 2, 3, 3, w, h))
    new_grid.append(new_grid_temp)
printGrid(new_grid)

..........
..........
..........
..........
.......##.
.......###
.......###
..........
..........
..........


# Creating a function to package everything

In [205]:
def GameOfLife(w, h, iterations, frequency, A, B, C, initial_state):
    #Creating the grid
    grid = conwaysGameOfLife(w, initial_state)
    printGrid(grid)
    print('\n')
    
    counter = 0
    print_counter = 0
    while counter < iterations:  
        new_grid = []
        for i in range(0, h):
            new_grid_temp = []
            for j in range(0, w):
                new_grid_temp.append(cellAliveOrDead(grid, i, j, A, B, C, w, h))
            new_grid.append(new_grid_temp)
        grid = new_grid
        
        counter +=1
        print_counter += 1
        if frequency == print_counter:
            printGrid(grid)
            print('\n')
            print_counter = 0
    return grid

In [164]:
%%time
GameOfLife(10, 10, 1000000, 999999, 2, 3, 3, '...........#.........#.......................#..#........##.........#.#.............................')

..........
.#........
.#........
..........
.....#..#.
.......##.
........#.
#.........
..........
..........


..........
..........
..........
..........
.......#..
......#.#.
......#..#
.......##.
..........
..........


CPU times: total: 9min 12s
Wall time: 9min 13s


# Testing 3.in

In [149]:
with open('3.in', 'r') as file:
    lines = file.readlines()

In [169]:
grid128 = ''.join(lines[1:]).replace('\n', '')

In [170]:
lines[0]

'128 128 1000 10 1 4 2\n'

In [208]:
final_grid = GameOfLife(128, 128, 1000, 10, 1, 4, 2, grid128)

#...#.#..###.........#.##.#.##..#.##.#..###..###.#####.#.#.##.#..#####..##.#...#...#####..###.####.#.#######..##...##.#.##.#..##
.#..#...#.##..##...#.#....#...#.#..#.#...####.##.##.###.####..##..##.#.####.#.....#..#..##...#.####.###....##..##.....####.###..
..##....########...#####.##..####..##.#...#######.#.##.##.####..#..#......#.#.#.......#..###..####..#.....#.###.......#.#..##..#
.#..#.....#.##.#####.#.#..#..##.##...##.###...##....###..#.####.###..##..#####.#.###.#...####.#.##....#.#.#.##.#.#.#######..#..#
##.#..####.#..#.##.....#..#...##.#..#.#..##.###....#.##.#####..####...#..##.#.#.####.####..#.#..##.#.#.##..###.#.#......##..#..#
###..##.##..#...####.##.#..###..###.#####.#.##..##.#####.#.###...#.##.###..##.####.###..##...#####..#.##...#.....#.##.##...#.##.
#..#....#.##...###..#.....##.#....#.#....#.###.###.##.#####..#..##..##..##.......#..#.#..#.##.#.###...#...###..#.###...#..#.####
..####...#.#.#.#...##..##.###...........#.#..#.#..#...#...####.#.#.###.#.#..#..#.###.....###..##.

.......................................##.......#..............#..#...##........................................................
................................................#................#..............................................................
................................................................................................................................
........................................................#.........#..........................#.....................#............
..................................#.....................#..........#........................#..............#.......#............
...................................#..........#...........................................................#.....................
.............................................#............................................................................##....
...................................................................#...........................##

.......................................##.......#..............#..#...##........................................................
................................................#................#..............................................................
................................................................................................................................
........................................................#.........#..........................#.....................#............
..................................#.....................#..........#........................#..............#.......#............
...................................#..........#...........................................................#.....................
.............................................#............................................................................##....
...................................................................#...........................##

.......................................##.......#..............#..#...##........................................................
................................................#................#..............................................................
................................................................................................................................
........................................................#.........#..........................#.....................#............
..................................#.....................#..........#........................#..............#.......#............
...................................#..........#...........................................................#.....................
.............................................#............................................................................##....
...................................................................#...........................##

.......................................##.......#..............#..#...##........................................................
................................................#................#..............................................................
................................................................................................................................
........................................................#.........#..........................#.....................#............
..................................#.....................#..........#........................#..............#.......#............
...................................#..........#...........................................................#.....................
.............................................#............................................................................##....
...................................................................#...........................##

...

.......................................##.......#..............#..#...##........................................................
................................................#................#..............................................................
................................................................................................................................
........................................................#.........#..........................#.....................#............
..................................#.....................#..........#........................#..............#.......#............
...................................#..........#...........................................................#.....................
.............................................#............................................................................##....
...................................................................#...........................##

.......................................##.......#..............#..#...##........................................................
................................................#................#..............................................................
................................................................................................................................
........................................................#.........#..........................#.....................#............
..................................#.....................#..........#........................#..............#.......#............
...................................#..........#...........................................................#.....................
.............................................#............................................................................##....
...................................................................#...........................##

.......................................##.......#..............#..#...##........................................................
................................................#................#..............................................................
................................................................................................................................
........................................................#.........#..........................#.....................#............
..................................#.....................#..........#........................#..............#.......#............
...................................#..........#...........................................................#.....................
.............................................#............................................................................##....
...................................................................#...........................##

.......................................##.......#..............#..#...##........................................................
................................................#................#..............................................................
................................................................................................................................
........................................................#.........#..........................#.....................#............
..................................#.....................#..........#........................#..............#.......#............
...................................#..........#...........................................................#.....................
.............................................#............................................................................##....
...................................................................#...........................##

.......................................##.......#..............#..#...##........................................................
................................................#................#..............................................................
................................................................................................................................
........................................................#.........#..........................#.....................#............
..................................#.....................#..........#........................#..............#.......#............
...................................#..........#...........................................................#.....................
.............................................#............................................................................##....
...................................................................#...........................##

In [209]:
printGrid(final_grid)

.......................................##.......#..............#..#...##........................................................
................................................#................#..............................................................
................................................................................................................................
........................................................#.........#..........................#.....................#............
..................................#.....................#..........#........................#..............#.......#............
...................................#..........#...........................................................#.....................
.............................................#............................................................................##....
...................................................................#...........................##